In [35]:
import meshplot
import scipy as sp
import numpy as np
import igl
import meshplot as m
import gpytoolbox
import trimesh

In [36]:

def read_matrix_from_file(filename):
    with open(filename, 'r') as file:
        line = file.readline().strip()
        x, y, z = map(int, line.split())

        total_elements = x * y * z

        positions = np.zeros((total_elements, 3)) 
        s = np.zeros(total_elements)              

        for i in range(total_elements):
            positions[i] = list(map(float, file.readline().strip().split()))

        for i in range(total_elements):
            s[i] = float(file.readline().strip())

    return x, y, z, positions, s

filename = '../data/SDF_file.txt'
x, y, z, positions, s = read_matrix_from_file(filename)

print(f"Dimensions: {x}, {y}, {z}")

print("Positions (first 5 entries):")
for i in range(min(5, len(positions))):
    print(f"Position {i}: {positions[i]}")

print("S values (first 5 entries):")
for i in range(min(5, len(s))):
    print(f"S[{i}] = {s[i]}")

color = s 

meshplot.plot(positions, c = color, shading={"point_size": 0.1, "colormap": "tab10"})

Dimensions: 26, 34, 32
Positions (first 5 entries):
Position 0: [-0.713163 -0.934783 -0.868123]
Position 1: [-0.658815 -0.934783 -0.868123]
Position 2: [-0.604467 -0.934783 -0.868123]
Position 3: [-0.550119 -0.934783 -0.868123]
Position 4: [-0.495771 -0.934783 -0.868123]
S values (first 5 entries):
S[0] = 0.357732
S[1] = 0.357728
S[2] = 0.357721
S[3] = 0.357709
S[4] = 0.357695


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.033814…

In [37]:
np.min(positions, axis=0)

array([-0.713163, -0.934783, -0.868123])

In [38]:
np.max(positions, axis=0)

array([0.645533, 0.858696, 0.81666 ])

In [42]:
cube = trimesh.creation.box(extents=[2, 2, 2], transform=trimesh.transformations.translation_matrix([-1, -1, -1]))

# 获取顶点和面
vertices = cube.vertices
faces = cube.faces

faces

TrackedArray([[1, 3, 0],
              [4, 1, 0],
              [0, 3, 2],
              [2, 4, 0],
              [1, 7, 3],
              [5, 1, 4],
              [5, 7, 1],
              [3, 7, 2],
              [6, 4, 2],
              [2, 7, 6],
              [6, 5, 4],
              [7, 5, 6]], dtype=int64)

In [44]:
u, g = gpytoolbox.remesh_botsch(vertices, faces, 20, 0.1, True)
meshplot.plot(u, g)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.0, -1.…